# Prompt Engineering (on Llama-2-Chat model)

Note demonstrates some prompting techniques and and how they can be used to get more desired answers which can be more accurate as well. It also demonstrates how we can get responses in a specific form so we can parse for further processing and use LLM as a part of bigger system.


In [2]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",

                                         )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
def get_llama2_chat_reponse(prompt, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, temperature= 0.00001)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

## Zero Shot Prompting

In [9]:
prompt = '''
[INST]
Classify the text into neutral, negative or positive.
Text: I just love it
[/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=100))


[INST]
Classify the text into neutral, negative or positive. 
Text: I just love it
[/INST]
Classification: Positive


In [ ]:
prompt = '''
[INST]
what is the hindi translation of the text
Text: I like eating
[/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=100))


[INST]
what is the hindi translation of the text 
Text: I like eating
[/INST]
The Hindi translation of the text "I like eating" is:

मैं खाना पसंद करता हूँ (Main khana pasand karta hoon)


## Few Shot Prompting

In [ ]:
prompt = '''
[INST] To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses
the word farduddle is:  [/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=200))


[INST] To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses
the word farduddle is:  [/INST]
I apologize, but "farduddle" is not a commonly recognized word in English. It is possible that it is a misspelling or a made-up word.

If you meant to use the word "fidget," which means to move your body around quickly and restlessly, you could use the sentence:

"She was fidgeting with excitement during the concert."

Alternatively, if you meant to use a different word, please let me know and I'll be happy to help you with your sentence.


In [ ]:
prompt = '''
[INST] A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses
the word whatpu is: [/INST]
We were traveling in Africa and we saw these very cute whatpus.
[INST] To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses
the word farduddle is: [/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=200))


[INST] A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses
the word whatpu is: [/INST]
We were traveling in Africa and we saw these very cute whatpus.
[INST] To do a "farduddle" means to jump up and down really fast. An example of a sentence that uses
the word farduddle is: [/INST]
The children were having a farduddle competition in the playground.


## Few Shot Prompting for Structure




In [ ]:
prompt = '''
[INST] <<SYS>>
Classify the text into neutral, negative or positive.
<</SYS>>

I like this.[/INST]
positive
[INST] I hate this [/INST]
negative
[INST] this is ok [/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=200))


[INST] <<SYS>>
Classify the text into neutral, negative or positive.
<</SYS>>

I like this.[/INST]
positive
[INST] I hate this [/INST]
negative
[INST] this is ok [/INST]
neutral


## Chain of Thought Prompting

In [ ]:
prompt = '''
[INST] True or False. The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82. [/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=200))


[INST] True or False. The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82. [/INST]

False.

The sum of the odd numbers in the group is 15 + 32 + 5 = 52, which is an odd number.


In [ ]:
prompt = '''
[INST] The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1. [/INST]
A: Odd numbers in the group are: 9, 15, 1. Sum of them is 25. 25 is odd. The answer is False.
[INST] The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82. [/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=200))


[INST] The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1. [/INST]
A: Odd numbers in the group are: 9, 15, 1. Sum of them is 25. 25 is odd. The answer is False.
[INST] The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82. [/INST]
A: Odd numbers in the group are: 15, 5, 13. Sum of them is 33. 33 is odd. The answer is False.


 Side note: LLMs should ideally not be used for numbers/calculations. Agent based system can be used where LLM has access to an agent (like calculator) to perform calculations.

 Also larger models can be used to improve accuracy.

In [ ]:
prompt = '''
[INST] True or False?
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82.
Let's think step by step. [/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=200))


[INST] True or False?
The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82. 
Let's think step by step. [/INST]

True.

To find the sum of the odd numbers in the group, we can add up the individual odd numbers:

15 + 32 + 5 + 13 = 63

Since 63 is an even number, the sum of the odd numbers in the group is even.


## Prompting for Chat

In [ ]:
prompt = '''
[INST] <<SYS>>
Respond only by quoting from the TV series Friends
<</SYS>>

Hi[/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=200))


[INST] <<SYS>>
Respond only by quoting from the TV series Friends
<</SYS>>

Hi[/INST]
"How you doin'?"


In [ ]:
prompt = '''
[INST] <<SYS>>
Respond only by quoting from the TV series Friends
<</SYS>>

Hi[/INST]
"How you doin'?"
[INST] whats the capital of India?[/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=200))


[INST] <<SYS>>
Respond only by quoting from the TV series Friends
<</SYS>>

Hi[/INST]
"How you doin'?"
[INST] whats the capital of India?[/INST]
"Oh, India? *thinks* Let me see... *pauses* Oh, I know this one! *smiling* The capital of India is... *drumroll* Delhi! *wink*"


#### Another example for Chat

In [ ]:
prompt = '''
[INST] <<SYS>>
Respond only by using emojis
<</SYS>>

Hi[/INST]
👋
[INST] whats the capital of India?[/INST]
🇮🇳
[INST] show me some dance move [/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=200))


[INST] <<SYS>>
Respond only by using emojis
<</SYS>>

Hi[/INST]
👋
[INST] whats the capital of India?[/INST]
🇮🇳
[INST] show me some dance move [/INST]
💃🕺


In [ ]:
prompt = '''
[INST]
Hi, my name is Yash and I am 27[/INST]
Hello Yash
[INST] whats the capital of India?[/INST]
Delhi
[INST] whats my name? [/INST]
Your name is Yash.
[INST] what is my age?[/INST]
'''
print(get_llama2_chat_reponse(prompt, max_new_tokens=200))


[INST] 
Hi, my name is Yash and I am 27[/INST]
Hello Yash
[INST] whats the capital of India?[/INST]
Delhi
[INST] whats my name? [/INST]
Your name is Yash.
[INST] what is my age?[/INST]
You are 27 years old.
